In [ ]:
#getting data from kaggle
import kaggle
!kaggle datasets download ankitbansal06/retail-orders -f orders.csv #data source

In [ ]:
#extracting data
import zipfile
zip_ref = zipfile.ZipFile('orders.csv.zip')
zip_ref.extractall()
zip_ref.close()

In [7]:
#read data
import pandas as pd
df = pd.read_csv(r"C:\Users\ariiq\orders.csv")

In [11]:
#checking, there is multiple nan values rows with different names
df['Ship Mode'].unique()

array(['Second Class', 'Standard Class', 'Not Available', 'unknown',
       'First Class', nan, 'Same Day'], dtype=object)

In [15]:
#handle multiple null values, renaming it to same name
df = pd.read_csv(r"C:\Users\ariiq\orders.csv", na_values=['Not Available', 'unknown'])

In [17]:
#check after
df['Ship Mode'].unique()

array(['Second Class', 'Standard Class', nan, 'First Class', 'Same Day'],
      dtype=object)

In [19]:
#renaming columns, lowering case and change " " to "_"
df.columns = df.columns.str.lower() #lower case

In [25]:
df.columns = df.columns.str.replace(' ', '_')

In [27]:
df.head()

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5


In [37]:
#derive new / more useful columns: dicount, sale price, profit
df['discount'] = df['list_price'] * df ['discount_percent'] * 0.01

In [43]:
df['sale_price'] = df['list_price'] - df ['discount']

In [49]:
df['profit'] = df['sale_price'] - df['cost_price']

In [ ]:
#check
df.head(5)

In [55]:
#convert order date columns data type from object to datetime, more accurate data type
#df.dtypes #check first
df['order_date'] = pd.to_datetime(df['order_date'], format='%Y-%m-%d')

In [57]:
df.dtypes

order_id                     int64
order_date          datetime64[ns]
ship_mode                   object
segment                     object
country                     object
city                        object
state                       object
postal_code                  int64
region                      object
category                    object
sub_category                object
product_id                  object
cost_price                   int64
list_price                   int64
quantity                     int64
discount_percent             int64
discount                   float64
sale_price                 float64
profit                     float64
dtype: object

In [67]:
#drop unnecessary columns
df.drop(columns = ['list_price', 'cost_price', 'discount_percent'], inplace = True) #use inplace to permanently

In [95]:
from sqlalchemy import create_engine
connection_string = 'mssql+pyodbc://Ariiq/ariiqdb?driver=ODBC+Driver+17+for+SQL+Server&Trusted_Connection=yes'
engine = create_engine(connection_string)
conn = engine.connect()

#Breakdown:
#Ariiq = server name
#ariiqdb = database name
#ODBC+.... = driver name, check odbc data source on your local

In [97]:
df.to_sql('df_orders', con = conn, index = False, if_exists = 'replace')

38

In [87]:
#After importing the data into SQL, the table structure appears messy. Therefore, we will create a new empty table in SQL.
#To do this, we'll first drop the existing table, create the new table with the appropriate structure (including the primary key column),
#and then append the data from here into the new table."

In [105]:
df.columns

Index(['order_id', 'order_date', 'ship_mode', 'segment', 'country', 'city',
       'state', 'postal_code', 'region', 'category', 'sub_category',
       'product_id', 'quantity', 'discount', 'sale_price', 'profit'],
      dtype='object')

In [107]:
df.to_sql('df_orders', con = conn, index = False, if_exists = 'append')
# we change the replace to append so it will append the content

38